## Luck vs. Skill (US Large Funds Edition)

Goal of this notebook is to test out the tools I made in the python file `luck_v_skill.py`, as well as try out some new, US data. This uses a 4 factor KAPM model to evaluate alphas.

In [57]:
import sys
sys.path.insert(0, '../src')

from luck_v_skill import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
# let's me save a jupyter notebook from a code cell
from IPython.display import Javascript

script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''

def notebook_save():
    Javascript(script)
    print('This notebook has been saved')

In [3]:
labels = pd.read_json('../data/US_funds_colrow_names.json')
labels = labels.sort_values(by='Start')
labels.Start -= 1 # pandas.read_excel() calls first row 0
labels

,Start,End
Columns,8,9
US OE Large Blend,10,949
US OE Large Value,951,1760
US OE Large Growth,1763,2726
US OE Mid-Cap Growth,2729,3179
US OE Small Value,3181,3485
US OE Mid-Cap Value,3488,3756
US OE Small Growth,3759,4262
US OE Small Blend,4265,4718
US OE Technology,4721,4835


#### Decide on Funds
For now, I'm only going to use large US fund data. I've laid out lists in order to compare mid-cap and small portfolios for later. Will still need to figure out how to drop the right rows when doing small or mid-cap.

In [4]:
# identify funds by description
large_names = list(labels.index[["Large" in fund_type for fund_type in labels.index]])
# mid_names = list(labels.index[["Mid-Cap" in fund_type for fund_type in labels.index]])
# small_names = list(labels.index[["Small" in fund_type for fund_type in labels.index]])

# decide which rows to skip
fund_types = large_names # + mid_names + small_names
skip_footer = labels.loc[fund_types,'End'].max()

# load data
US_funds = pd.read_excel('../data/US Equity Mutual Funds Monthly Returns.xlsx',
                         header=labels.loc['Columns','Start'], skipfooter=skip_footer)
# drop empty rows
empty_rows = (~US_funds.isna()).sum(axis=1) < 3
US_funds.drop(US_funds.index[empty_rows],axis=0,inplace=True)

# replace missing ticker names with fund name
no_tick = US_funds.index[US_funds.Ticker.isna()]
US_funds.iloc[no_tick,1] = US_funds.iloc[no_tick,0]
US_funds.set_index('Ticker',inplace=True)
US_funds.drop(columns=['Group/Investment','Fund Size'],inplace=True)

# transpose and convert to datetime
funds = US_funds.T
funds.index = pd.to_datetime(funds.index.str.strip('Monthly Return '))

# info
funds.shape

(240, 2712)

Load factors

In [5]:
momentum = pd.read_csv('../data/F-F_Momentum_Factor.CSV')
factors = pd.read_csv('../data/F-F_Research_Data_Factors.CSV')
# convert 
for df in [momentum,factors]:
    df['Dates'] = pd.to_datetime(df.Dates.astype(str)\
                                 .apply(lambda x: x[0:4]+'/'+x[4:]),
                                 format="%Y/%m")
    df.set_index('Dates',inplace=True)

factors = pd.merge(factors, momentum, left_index=True, right_index=True)
factors.shape

(1105, 5)

Line up time of data

In [6]:
fund_start,fund_end = funds.index.min(),funds.index.max()
fact_start,fact_end = factors.index.min(),factors.index.max()

# clip starts
if fact_start < fund_start:
    factors.drop(factors.index[factors.index < fund_start],inplace=True,axis=0)
else:
    funds.drop(funds.index[funds.index < fact_start],inplace=True,axis=0)
    
# clip ends
if fund_end > fact_end:
    funds.drop(funds.index[funds.index > fact_end],inplace=True,axis=0)
else:
    factors.drop(factors.index[factors.index > fund_end],inplace=True,axis=0)

# check shapes
funds.shape, factors.shape

((239, 2712), (239, 5))

In [59]:
# select columns of interest
# take out Mom for 3 factor KAPM
factor_cols = ['Mkt-RF','HML','SMB','RF', 'Mom'] 
factors = factors[factor_cols]

### Initial Simulation (standard deviation alpha = 0)

In [108]:
# This code cell runs the data through the AlphaEvaluator object found in
# the luck_v_skill.py file. Creates tables of actual alpha percentiles vs
# simulated alpha percentiles with increasing standard deviation asssumptions
# about injected alpha. The first operation assumes std_alpha = 0.

#----------------------------------------------------------------------------
# OUTPUT PARAMETERS
#----------------------------------------------------------------------------
top_n = 10 # look at top 10 and bottom ten funds, in addition to percentiles
pct_range = np.arange(1,10)/10 # percentiles to consider.  0 < x < 1
verbose = True # keep track of where in program we are at
min_obs = 120 # funds must have at least this many obs to be considered
sim_cutoff = 15 # minimum non-NaN obs in sim_matrix obs for regression
n_simulations = 1000 # number of simulations to run
random_seed = 2 # seed the random number generator here
std_range = np.arange(1,21)/20 # standard deviations of alpha to simulate
n_std = len(std_range) + 1 # number of extra standard deviations evaluating


#----------------------------------------------------------------------------
# START
#----------------------------------------------------------------------------
# Time the program
start_time = time.time()
if verbose:
    title_print("Standard Deviation {} of {}".format(1,n_std))

# initialize instance of the AlphaEvaluator with fund and factor data
AEV = AlphaEvaluator(fund_data=funds,factor_data=factors)

# calculates original coefficients, standard errors, and t-stats
AEV.fit(min_obs=min_obs)



# runs simulations and populates simulated coeff, SE, and tstat matrices
AEV.simulate(n_simulations=n_simulations,random_seed=random_seed,sim_std=0,
             sim_cutoff=sim_cutoff,verbose=verbose)

# populates percentile tables for alpha and t(alpha)
AEV.get_percentiles(pct_range=pct_range,verbose=verbose,top_n=top_n)
data_a, data_t = AEV.data_a.copy(), AEV.data_t.copy()

# Rename data columns
data_a.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']
data_t.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']

--------------------------
Standard Deviation 1 of 21
--------------------------
Annual standard deviation: 0.00, Monthly standard deviation alpha: 0.00
Starting 2,703,000 regressions...


KeyboardInterrupt: 

### Plots

In [ ]:
fig, axes = AEV.plot(plot_type=['cdf','kde','hist'],
                     statistic=['alpha','t-stat'],
                     fund=[-1,0])
plt.tight_layout()
plt.savefig('../charts/US-Large-4factor-plots.png')

### More Simulations

In [ ]:
# fill data with more simulations
for i,stdev in enumerate(std_range):
    if verbose:
        title_print("Standard Deviation {} of {}".format(i+2,n_std))
    
    # Rerun simulation with new standard deviation injection for alpha
    AEV.simulate(n_simulations=n_simulations,random_seed=random_seed,
                 sim_std=stdev,sim_cutoff=sim_cutoff,verbose=verbose)
    
    # Update Percentile Data
    AEV.get_percentiles(pct_range=pct_range,verbose=True,top_n=top_n)
    
    # Populate Alpha Percentile Table
    data_a['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_a['Sim Avg']
    data_a['%<Act ({:.2f})'.format(stdev)] = AEV.data_a['%<Act']
    
    # Populate t-statistics Percentile Table
    data_t['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_t['Sim Avg']
    data_t['%<Act ({:.2f})'.format(stdev)] = AEV.data_t['%<Act']


#----------------------------------------------------------------------------
# END
#----------------------------------------------------------------------------

# calculate time elapsed for program and print    
t_diff = time.time()-start_time
minutes = int(np.floor(t_diff/60))
seconds = t_diff - 60*minutes
if verbose:
    print("DONE!")
    print("{} minutes and {:.2f} seconds elapsed for this program".format(minutes,seconds))

In [ ]:
data_a.to_csv('../tables/US-Large-4factor-alphas.csv')
data_t.to_csv('../tables/US-Large-4factor-tstats.csv')

In [9]:
data_a.shape, data_t.shape

((29, 43), (29, 43))

In [10]:
notebook_save()

This notebook has been saved
